## Exploratory Analysis

That's it for some preliminary cleaning. Don't worry, there will be more. Let's start to look in a bit more detail at the data, though. In this section, we're going to start to write some code that's typical for day-to-day data cleaning tasks.

In [ ]:
from load_data import dta

We can use `info` to get some high-level information about the data.

In [ ]:
dta.info()

And `describe` goes into a bit more detail for the *numeric* types, of which we don't have many here.

In [ ]:
dta.describe()

We could do the same for the categorical types.

In [ ]:
dta.select_dtypes(['category']).describe()

### GroupBy

Now, let's ask the most obvious question. Which are the best and the worst restaurants? We'll want to use pandas `GroupBy` functionality to implement the `split-apply-combine` pattern.

The idea here is that we **split** the data by some key or set of keys then **apply** a function to each group and then **combine** the outputs back into a single DataFrame.

First, let's see how many result categories there are. We can use `value_counts` to answer this question. 

In [ ]:
with pd.option_context("max.rows", 10):
    print(dta.results.value_counts())

Ok, let's group on the inspection `results` column and see who the best and worst are.

When we call the `groupby` method we get back a `DataFrameGroupBy` object.

In [ ]:
grouper = dta.groupby(dta.results)

In [ ]:
grouper

You can access the variables on this object, the same as a DataFrame, and any code called will execute within the groups.

In [ ]:
result = grouper.dba_name.value_counts()

The result is a Series with a `MultiIndex`.

In [ ]:
result

In [ ]:
result.index.names

We can index on the first element in a `MultiIndex` using square brackets and then use `sort_values` to find those restaurants that had a result of Fail the most.

In [ ]:
with pd.option_context('max.rows', 15):
    print(result["Fail"].sort_values(ascending=False))

Take a closer look above. Looks like we have some more data cleaning to do.

In [ ]:
with pd.option_context('max.rows', 15):
    print(result["Pass"].sort_values(ascending=False))

Obviously, this is probably not the right way to think about this since there are many more Subways than local establishments.

We could instead look at the ratio of Fail to Pass. 


Sometimes, it's not *always* obvious how to go about computing things that you want to compute. The `get_group` method allows you to pull out one of the split DataFrames and try your apply function on it.

In [ ]:
grouper = dta.groupby(dta.dba_name)

mcd = grouper.get_group("MCDONALD'S")

We can write our relative counts function like so and test it out on the data.

In [ ]:
def relative_results(df):
    values = df.value_counts()
    return values['Fail'] / values['Pass']

And we see McDonald's failed 50% as many inspections as it Passed.

In [ ]:
relative_results(mcd.results)

We can run this on everything, but it's going to be a little slow. Let's look at a different solution. Here group by *both* the inspection results and the DBA name. Then we ask for the `size` of each one of these groups.

In [ ]:
result = dta.groupby((dta.results, dta.dba_name)).size()

In [ ]:
result

We can use the `div` method to divide these for us. As you can see the indices don't line up, but we don't have to worry about it. Pandas take care of index alignment for us.

In [ ]:
result["Fail"]

In [ ]:
result["Pass"]

In [ ]:
ratio = result["Fail"].div(result["Pass"])
ratio.sort_values(ascending=False, inplace=True)
ratio

We have a lot of `NaN`s in the results from division-by-zero. We can drop those with a call to `dropna`. Also note that pandas lets you decide whether to treat `inf` as an NA.

In [ ]:
with pd.option_context("use_inf_as_null", True,
                       "max.rows", 15):
    print(ratio.dropna())

We might still not be wholly satisfied with our rules around comparisons here. First, we're looking at restaurant names not particular establishments. What does the distribution of inspection visits for establishments look like.

In [ ]:
with pd.option_context("max.rows", 10):
    grouper = dta.groupby((dta.address, dta.dba_name))
    print(grouper.size().describe())

Let's see what the Fail:Pass ratio is for restaurants with at least 3 visits that involved a high risk level. 


Now we're starting to get into some much more powerful pandas constructs.

In [ ]:
def rename(series, name):
    series.name = name
    return series

In [ ]:
visited = (dta.query("risk == 'Risk 1 (High)'")
           .groupby(('address', 'dba_name'))
           .size()
           .pipe(lambda df: rename(df, 'n_visits'))  # size returns a nameless series
           .reset_index()  # make into a DataFrame
           .query("n_visits >= 4"))

visited

Let's unpack this. The first thing to note is how this code is organized. Each one of these methods return a pandas data structure on which we call the next method. This is called **method chaining**. We use the same trick seen above to split strings across lines to split several method calls by including the code between `()`.

Next, we see several new methods. The first is **query**. When subsetting a DataFrame we have a few options. As we save above, we can index a DataFrame using integers. Likewise, we could pass an object of booleans as well.

In [ ]:
dta.risk == "Risk 1 (High)"

In [ ]:
dta.loc[dta.risk == "Risk 1 (High)"]

Always using indexing can be verbose, however. You may need compound statements, for example.

In [ ]:
dta.loc[(dta.risk == "Risk 1 (High)") | (dta.risk == "Risk 1 (Medium)")].head()

Instead, by using query we could write the following, which is slightly easier.

In [ ]:
dta.query("(risk == 'Risk 1 (High)') | (risk == 'Risk 1 (Medium)')").head()

```python
visited = (dta.query("risk == 'Risk 1 (High)'")
           .groupby(('address', 'dba_name'))
           .size()
           .pipe(lambda df: rename(df, 'n_visits'))  # size returns a nameless series
           .reset_index()  # make into a DataFrame
           .query("n_visits >= 4"))
```


The next new method is the **pipe** method. This allows for including user-defined functions in method chains. It takes the output of the thing on the left, and passes it to the thing on the right. Here we're renaming the series, so it's readable later what we're doing. Readability counts.

Finally, we filter on restaurants with 4 or more total visits.

The final piece is computing the Fail:Pass ratio of these restaurants. To do this, we need to take the output we've created and restrict the original non-aggregated data to it. We can do this by executiong a **merge**. By default, the `merge` method will join together two DataFrames on common columns, using an inner join method (a set intersection).

In [ ]:
(visited.merge(dta)
 .groupby(('results', 'address', 'dba_name'))
 .size()
 .pipe(lambda df: df['Fail'].div(df['Pass']))
 .dropna()
 .sort_values(ascending=False))

We can put all this together.

In [ ]:
with pd.option_context('max.rows', 15):
    print(dta.query("risk == 'Risk 1 (High)'")
          .groupby(('address', 'dba_name'))
          .size()
          .pipe(lambda df: rename(df, 'n_visits')
          .reset_index()
          .query("n_visits >= 4")
          .merge(dta)
          .groupby(('results', 'address', 'dba_name'))
          .size()
          .pipe(lambda df: df['Fail'].div(df['Pass']))
          .dropna()
          .sort_values(ascending=False)))

Often, I'll try to place a method chain to get the data ready for more exploratory work at the top of a notebook, so I can proceed with any analyses. Let's fold back in

Now, let's go back and add in the unstacked violations. Recall that we unstack the violations as follows. There are two new things to note here though. We add in a `to_frame` method to turn the unstacked Series into a DataFrame, and we `rename` the unnamed column in the resulting DataFrame back to `violations`.

In [ ]:
(dta.violations
 .str.split("|", expand=True)
 .unstack()
 .dropna()
 .reset_index(level=0, drop=True)
 .str.strip()
 .to_frame()
 .rename(columns={0: 'violations'}))

First, we need to drop the violations from the original DataFrame, then we need to merge it with the unstacked violations Series that we created before.

In [ ]:
dta.drop(["violations"], axis='columns').head()

Here we drop the original violations from the DataFrame, unstack them as before, turn this result into a DataFrame for joining, rename the unnamed column to `violations` and perform a **right join**.

We use **join** here rather than merge. Join uses merge under the hood but conveniently allows us to join on the indices of the two DataFrames by default. One other difference is that join uses an inner merge by default, but that's not what we want here. Since we drop the null violations on the right-hand side DataFrame, we want to do a right join. 

In [ ]:
dta = dta.drop(["violations"], axis='columns').join(
    dta.violations.str.split("|", expand=True)
        .unstack()
        .dropna()
        .str.strip()
        .reset_index(level=0, drop=True)
        .to_frame()
        .rename(columns={0: 'violations'}),
    how='right'
)

In [ ]:
dta.head()

Now that we have a relatively clean DataFrame, let's ask a few more questions. 

First, how many unique violations do we have?

In [ ]:
dta.shape

In [ ]:
dta.violations.head()

In [ ]:
dta.violations.unique().shape

Is this true? Do we really think there are this many violation numbers? Probably not. We can use the `str` accessor and some more munging to answer this. Here we pass a **regular expression** to `str.extract`. Extract expects a *capture group*, indicated by `()`. The regular expression `(\d+\)(?=\.)` means capture 1 or more (`+`) digits (`\d`) that is followed by (`(?=)`) a period `\.`. We escape the period because a plain `.` is a wildcard for any character.

In [ ]:
(dta.violations
 .str.extract("(\d+)(?=\.)", expand=False)
 .astype(int))

In [ ]:
np.sort(
    dta.violations
        .str.extract("(\d+)(?=\.)", expand=False)
        .astype(int)
        .unique()
).shape

Second, can we figure out how many times previous did an establishment fail an inspection (within the sample we have)?

In [ ]:
visits = dta.drop_duplicates(["address", "dba_name", "inspection_date"])

We need to ensure that the inspection dates are sorted within each group. GroupBy will preserve this.

In [ ]:
visits = visits.sort_values(["address", "dba_name", "inspection_date"])

In [ ]:
grouper = visits.groupby((visits.address, visits.dba_name))

Ok, we might ask, "now what?" Remember the trick to pull out groups?

In [ ]:
group_key = list(grouper.groups.keys())[0]

In [ ]:
group_key

In [ ]:
group = grouper.get_group(group_key)

In [ ]:
group

In [ ]:
group[['inspection_date', 'results']]

In [ ]:
group.merge()

Since, we need this to be backwards looking, we have to **shift** the data by one visit. Shifting will move the data around by either a number of periods or a frequency. In this case, we use a number of periods and shift forward by 1 period.

In [ ]:
group.shift(1)[['inspection_date', 'results']]

If we take the cumulative sum of this, we'll have an accurate picture of previous failures.

In [ ]:
(group.shift(1).results == 'Fail').cumsum()

In [ ]:
visit_num = grouper.apply(lambda df: (df.shift(1).results == 'Fail').cumsum())

In [ ]:
visit_num.head(n=15)

In [ ]:
(visit_num.reset_index(level=[0, 1], drop=True)
 .to_frame()
 .rename(
    columns={'inspection_date': 'num_fails'}
))

In [ ]:
dta.join((visit_num.reset_index(level=[0, 1], drop=True)
 .to_frame()
 .rename(
    columns={'inspection_date': 'visit_number'}
)))